## True AlexNet implementation with generator and Grayscale images
Each image 320x240x1

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1790062637444749394, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 356122624
 locality {
   bus_id: 1
 }
 incarnation: 14714242894333997388
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"]

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from sklearn.utils import shuffle
from keras import backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
width, height = 320, 240
train_data_dir = '../gen_x3/train'
test_data_dir = '../gen_x3/test'
nb_train_samples = 47232
nb_test_samples = 11808
epochs = 20
batch_size = 64

input_shape = (width, height, 1)

In [4]:
model = Sequential()
model.add(Conv2D(96, kernel_size=(15, 15), strides=(4, 4), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(9, 9), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dense(6, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
# summarize model and apply checkpoints
print(model.summary())
filepath = 'generator2_True_AlexNet-{epoch:02d}-{loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 77, 57, 96)        21696     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 28, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 20, 256)       1990912   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 10, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              78645248  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
__________

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.20,
    zoom_range=0.15,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(width, height),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(width, height),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    classes=['1', '2', '3', '4', '5', '6'],
    shuffle=True,
    seed=2018)

Found 47232 images belonging to 6 classes.
Found 11808 images belonging to 6 classes.


In [9]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks_list,
    shuffle=True,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

Epoch 1/20
738/738 [==============================] - 239s 324ms/step - loss: 1.8085 - acc: 0.1655 - val_loss: 1.7918 - val_acc: 0.1668
Epoch 2/20
738/738 [==============================] - 232s 315ms/step - loss: 1.7919 - acc: 0.1616 - val_loss: 1.7918 - val_acc: 0.1666
Epoch 3/20
738/738 [==============================] - 233s 316ms/step - loss: 1.7919 - acc: 0.1653 - val_loss: 1.7918 - val_acc: 0.1668
Epoch 4/20
738/738 [==============================] - 232s 315ms/step - loss: 1.7919 - acc: 0.1627 - val_loss: 1.7918 - val_acc: 0.1664
Epoch 5/20
738/738 [==============================] - 232s 315ms/step - loss: 1.7919 - acc: 0.1646 - val_loss: 1.7918 - val_acc: 0.1668
Epoch 6/20
738/738 [==============================] - 232s 315ms/step - loss: 1.7918 - acc: 0.1638 - val_loss: 1.7918 - val_acc: 0.1668
Epoch 7/20
738/738 [==============================] - 233s 315ms/step - loss: 1.7919 - acc: 0.1631 - val_loss: 1.7918 - val_acc: 0.1666
Epoch 8/20
738/738 [============================

In [ ]:
score = model.evaluate_generator(test_generator)
print("Accuracy: %.2f%%" % (score[1]*100))
print('Test loss:', score[0])
print('Test accuracy', score[1])

In [ ]:
model_json = model.to_json()
with open('generator2_True_AlexNet_model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('generator2_True_AlexNet_weights.h5')
print('Saved model weights')

In [ ]:
#summarize for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#summarize for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()